In [9]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from gensim.utils import simple_preprocess
import spacy

nlp = spacy.load('fr_core_news_sm')

# Fonction pour prétraiter le texte
def preprocess(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]


# Charger le modèle et le tokenizer BERT
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Fonction pour obtenir les embeddings de phrase
def get_sentence_embedding(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Documents à classifier
documents = [
    """La théorie astronomique des paléoclimats est un modèle utilisé en sciences de la Terre pour expliquer le caractère cyclique des variations générales du climat au cours des temps géologiques. Cette théorie explique que l'énergie radiative reçue par la Terre en provenance de l'espace — essentiellement du Soleil — est quantitativement affectée de variations à long terme, corrélées à celles de son orbite et de ses paramètres de rotation. Les variations concernent la quantité nette d'énergie reçue, sa répartition et sa « gestion » par l'écosystème Terre. La théorie avance que ces variations sont périodiques, et qu'elles influencent par conséquent le climat selon des motifs cycliques (alternance de périodes glaciaires et interglaciaires, notamment). Il est possible d'en retrouver une trace via différents indicateurs, par exemple dans les roches sédimentaires ou des carottages de glace, dont l'étude fait partie de la cyclostratigraphie. La théorie astronomique des paléoclimats est aujourd'hui l'explication classique des phénomènes de (dé)glaciation au Quaternaire, mais trouve également des applications, moins assurées, pour des périodes plus anciennes de l'échelle des temps géologiques.""",
    """Les plans d'organisation du vivant et la phylogénie sont des concepts clés en biologie qui permettent de comprendre la diversité et l'évolution des organismes. Le plan d'organisation fait référence à l'architecture générale et aux caractéristiques structurelles d'un organisme, incluant la disposition des tissus, des organes et des systèmes organiques. Ces plans peuvent varier considérablement entre les groupes d'organismes, reflétant leur adaptation à des environnements spécifiques et leurs modes de vie particuliers. La phylogénie, quant à elle, est l'étude des relations évolutives entre les espèces. Elle est représentée sous forme d'arbres phylogénétiques, qui illustrent les lignées évolutives et les points de divergence entre les différentes espèces. Ces arbres sont construits en analysant des données morphologiques, génétiques et moléculaires, permettant ainsi de retracer l'histoire évolutive des organismes et de comprendre comment les différents plans d'organisation sont apparus et ont évolué au fil du temps. L'intégration des concepts de plan d'organisation et de phylogénie offre une vision complète de la biodiversité. Elle permet d'identifier les traits ancestraux et les innovations évolutives qui ont conduit à la diversité actuelle des formes de vie. Par exemple, l'étude des vertébrés révèle des plans d'organisation complexes avec des structures telles que la colonne vertébrale, qui sont apparues chez un ancêtre commun et ont été modifiées dans divers groupes de descendants.""",
    """Lecture zen Après avoir rappelé ce que signifie « classer les êtres vivants », quels en sont et quels en ont été, historiquement, les objectifs, cet article présente les principes de la classification phylogénétique moderne. Il présente également, pour le niveau scolaire primaire, une façon de classer n'utilisant pas de données évolutives, mais compatible avec elles. Il s'accompagne d'une bibliographie, d'un glossaire et de mises au point relatives à plusieurs idées fausses. 1.    Introduction générale On pourrait penser que classer les êtres vivants est une activité maîtrisée des chercheurs en systématique (science des classifications), des naturalistes amateurs, des enseignants et des utilisateurs de la nature (agriculteurs, sélectionneurs, agronomes, pharmaciens, vétérinaires, paysagistes, conservateurs de parcs et de musées, etc.). Il n’en est rien. Même chez les scientifiques dont le métier utilise des classifications, il arrive encore trop souvent qu’on se méprenne sur ce qu’est que « classer ». Ceci est peut-être dû au fait qu’on a tous une intuition de ce en quoi consiste cette activité. Il peut paraître inutile alors de rappeler les fondements de l’activité classificatoire en général, et plus particulièrement en sciences naturelles. Faire des livres sur un aspect aussi fondamental des sciences semblerait aussi futile qu’enfoncer des portes ouvertes. C’est mal connaître le contexte. Récemment, à l’occasion d’un changement de programme, beaucoup d’enseignants du secondaire découvrirent qu’ils n’avaient jamais classé, et que ce que les programmes et les livres scolaires présentaient comme de la « classification » n’en était pas...""",
    """Prenons par exemple le concept de « fruits de mer ». Dans la nature il n’y a pas de fruits de mer. Il n’y a que des individus. « Fruits de mer » est un concept très sophistiqué. C’est un concept culinaire. On a mis dans un ensemble, sous ce nom, une cinquantaine d’espèces parce qu’elles constituent une palette gustative homogène. On ne choisit d’ailleurs pas n’importe quel vin avec un plateau de fruits de mer. Dans la nature il n’y a pas d’oiseaux. Il n’y a que des individus. « Oiseaux » est un concept très sophistiqué. C’est un concept phylogénétique. On a mis dans un ensemble plus de 10 000 espèces parce qu’elles portaient toutes des plumes, un bréchet, une fourchette, un bec, un hallux retourné, un pygostyle. Tous ces traits ont été acquis par voie d’ascendance commune, ce qui fait que le concept « oiseau » est phylogénétique. La classification construit des ensembles - des concepts - emboîtés les uns dans les autres. Au sein des oiseaux, il y a les psittaciformes, les falconiformes, etc. Encore une fois, la classification n’a pas pour vocation d’être pratique. Elle nous parle du monde qui existe, selon un « cahier des charges » qui a été fixé, et qui édicte la nature des concepts qui sont créés. C’est tout le contraire de la clé de détermination, qui, elle, a pour vocation d’être pratique et ne dit rien sur le monde. Il existe des naturalistes qui préfèrent les « anciennes classifications » parce qu’ils les trouvent « pratiques ». En fait, il ne s’agit pas de classifications aujourd’hui valides, mais d’un double mélange (Fig. 4), d’une part entre échelle des êtres et phylogénie, d’autre part d’un mélange de multiples cahiers des charges, linnéen typologistes (qui relève de concepts d’un autre temps), écologico-adaptatifs (qui relève de concepts différents) et phylogénétiques (qui relèvent de concepts modernes que les classificateurs ont décidé de créer). Ces figures émanaient d’une procédure où les classes linnéennes étaient prises comme des existants et l’on se mettait à chercher alors les « affinités évolutives » entre ces classes. Dans la procédure moderne, on cherche d’abord les affinités phylogénétiques entre les espèces et on les classe seulement ensuite. On reproche parfois aux scientifiques de compliquer la classification, de faire des classifications « pas pratiques ». On exige parfois qu’une classification donnée soit pratique parce qu’on n’a pas compris que c’est elle qui créé les concepts. On n’entend personne dire que le nombre d’objets astronomiques ne rend pas « pratique » la compréhension de l’univers, ni qu’il n’est pas « pratique » d’avoir plus de 100 éléments dans la table de Mendeleïev. Le monde est ce qu’il est ; la science doit en rendre compte rationnellement et ne peut dans son exercice subir les contraintes de l’utilisation des connaissances. En revanche, dans un but pédagogique, on peut toujours simplifier une classification scientifique."""
]

# Catégories à prédire
categories = [
    "Enregistrements des paléoclimats et des phénomènes tectoniques",
    "se situer dans l'espace et dans le temps (SVT cycle 4)",
    "comprendre les responsabilités individuelle et collective en matière de préservation des ressources de la planète",
    "La cellule, unité structurale et fonctionnelle du vivant",
    "L’organisme, une société de cellules",
    "Plans d’organisation du vivant et phylogénie",
    "random category random category random category"
]

# Prétraiter les documents et les catégories
preprocessed_documents = [" ".join(preprocess(doc)) for doc in documents]
preprocessed_categories = [" ".join(preprocess(cat)) for cat in categories]

# TF-IDF vectorisation
vectorizer = TfidfVectorizer()
all_texts = preprocessed_documents + preprocessed_categories
vectorizer.fit(all_texts)
tfidf_documents = vectorizer.transform(preprocessed_documents)
tfidf_categories = vectorizer.transform(preprocessed_categories)
tfidf_similarity_matrix = cosine_similarity(tfidf_documents, tfidf_categories)

document_embeddings = np.array([get_sentence_embedding(doc, tokenizer, model) for doc in documents])
category_embeddings = np.array([get_sentence_embedding(cat, tokenizer, model) for cat in categories])
bert_similarity_matrix = cosine_similarity(document_embeddings, category_embeddings)

combined_similarity_matrix = 5 * tfidf_similarity_matrix + 0.5 * bert_similarity_matrix

for doc_idx, doc in enumerate(documents):
    print("==================")
    print(f"Document: {doc[:100]}...")
    similarities = combined_similarity_matrix[doc_idx]
    for cat_idx, category in enumerate(categories):
        tfidf_similarity = tfidf_similarity_matrix[doc_idx, cat_idx]
        bert_similarity = bert_similarity_matrix[doc_idx, cat_idx]
        final_similarity = tfidf_similarity * 0.5 + bert_similarity * 0.5
        print(f"Category: {category[:30]}, TF-IDF Similarity: {tfidf_similarity:.4f}, BERT Similarity: {bert_similarity:.4f}, Final Similarity: {final_similarity:.4f}")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Document: La théorie astronomique des paléoclimats est un modèle utilisé en sciences de la Terre pour explique...
Category: Enregistrements des paléoclima, TF-IDF Similarity: 0.1033, BERT Similarity: 0.6982, Final Similarity: 0.4007
Category: se situer dans l'espace et dan, TF-IDF Similarity: 0.0700, BERT Similarity: 0.6533, Final Similarity: 0.3616
Category: comprendre les responsabilités, TF-IDF Similarity: 0.0000, BERT Similarity: 0.6073, Final Similarity: 0.3036
Category: La cellule, unité structurale , TF-IDF Similarity: 0.0000, BERT Similarity: 0.6209, Final Similarity: 0.3105
Category: L’organisme, une société de ce, TF-IDF Similarity: 0.0000, BERT Similarity: 0.5298, Final Similarity: 0.2649
Category: Plans d’organisation du vivant, TF-IDF Similarity: 0.0000, BERT Similarity: 0.5646, Final Similarity: 0.2823
Category: random category random categor, TF-IDF Similarity: 0.0000, BERT Similarity: 0.3625, Final Similarity: 0.1813
Document: Les plans d'organisation du vivant et la ph